In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import plotly.express as px

# Data Exploration

In [ ]:
data_1 = pd.read_csv("data/daily_financial_news/analyst_ratings_processed.csv", index_col=0)
data_1.head()

In [ ]:
data_1['date'] = data_1['date'].str.split(' ', expand=True).iloc[:, 0]

In [ ]:
data_1.head()

In [ ]:
data_1['stock'].nunique()

In [ ]:
data_1.info()

## Drop Null Values

In [ ]:
print(data_1.isna().sum())

In [ ]:
print(data_1.shape)

In [ ]:
data_1[data_1['date'].isna()]

In [ ]:
data_1.dropna(subset=['date'],inplace=True)
data_1.dropna(subset=['stock'],inplace=True)

In [ ]:
print(data_1.shape)

In [ ]:
print(data_1.isna().sum())

In [ ]:
data_1.info()

## Filter stocks

In [ ]:
# Filter dataset down to stocks with top 100 number of headlines
top_100_stocks_by_headlines = data_1.groupby('stock').size().reset_index(name='size').sort_values('size', ascending=False).reset_index(drop=True).iloc[:100]
top_100_stocks_by_headlines.head()

In [ ]:
data_1 = data_1[data_1['stock'].isin(top_100_stocks_by_headlines.stock)]

In [ ]:
data_1.shape

## Clean Text Data

In [ ]:
data_1['title'] = data_1['title'].str.lower()

In [ ]:
data_1.head()

### Remove Punctuations

In [ ]:
import string

def remove_punctuation(text):
    return ''.join([char for char in text if char not in string.punctuation and not char.isdigit()])

data_1['title'] = data_1['title'].apply(remove_punctuation)

In [ ]:
data_1.head()

In [ ]:
data_1['tokens'] = data_1['title'].apply(lambda x: x.split())
data_1.head()

### Remove stop words

In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

stop = stopwords.words('english')

data_1['tokens'] = data_1['tokens'].apply(lambda x: [word for word in x if word not in stop])

In [ ]:
data_1.shape

### Lemmatize tokens

In [ ]:
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

data_1['tokens'] = data_1['tokens'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])


In [ ]:
data_1.head()

In [ ]:
data_1['preprocessed_text'] = data_1['tokens'].apply(' '.join)
data_1.head()

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()
# NLTK Sentiment Intensity Analyzer uses a 'Bag of Words' approach
# it removes stop words and scores each word individually before compounding

In [ ]:
data_duplicate = data_1.copy()
data_duplicate['sentiment_score'] = data_duplicate['preprocessed_text'].apply(lambda x: sia.polarity_scores(x)['compound'])
data_duplicate

In [ ]:
data_duplicate['sentiment_score'].describe()

In [ ]:
data_duplicate.info()

# Pull yfinance data

In [95]:
tickers = data_duplicate.stock.unique()

In [100]:
start_date, end_date = data_duplicate.date.sort_values().iloc[0], data_duplicate.date.sort_values().iloc[-1]

In [105]:
data = yf.download(list(tickers), start=start_date, end=end_date)

[*********************100%%**********************]  100 of 100 completed


13 Failed downloads:
['AGN', 'JCP', 'ZNGA', 'MON', 'APC', 'BBBY', 'MYL']: Exception('%ticker%: No timezone found, symbol may be delisted')
['PCLN', 'LNKD', 'BBRY', 'GMCR']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2009-04-29 -> 2020-06-11)')
['CHK', 'SHLD']: Exception("%ticker%: Data doesn't exist for startDate = 1240977600, endDate = 1591848000")


In [106]:
data

Adj Close                                                        \
                    A        ADBE AGN        AIG APC       ATVI        AVGO   
Date                                                                          
2009-04-29  11.629640   26.750000 NaN  18.591227 NaN   9.411404         NaN   
2009-04-30  11.777991   27.350000 NaN  18.195669 NaN   9.446487         NaN   
2009-05-01  12.132751   27.459999 NaN  18.195669 NaN   9.507887         NaN   
2009-05-04  12.526214   26.690001 NaN  19.250494 NaN   9.692082         NaN   
2009-05-05  12.081153   26.889999 NaN  22.810509 NaN   9.621910         NaN   
...               ...         ...  ..        ...  ..        ...         ...   
2020-06-04  89.064148  385.799988 NaN  31.996899 NaN  68.326012  277.644440   
2020-06-05  88.321449  392.899994 NaN  33.041962 NaN  67.954094  285.005951   
2020-06-08  88.233482  397.779999 NaN  35.840897 NaN  69.226448  285.401489   
2020-06-09  87.725349  397.160004 NaN  34.296036 NaN  70.469452  283.424072   
2020-06-10  87.764427  406.820007 NaN  31.951466 NaN  72.250763  283.172394   

                                               ...   Volume           \
                   AXP        AZN        BABA  ...     URBN      VMW   
Date                                           ...                     
2009-04-29   20.127361  10.166079         NaN  ...  9728600  1814000   
2009-04-30   20.345181   9.726615         NaN  ...  9012600  2106300   
2009-05-01   19.594938   9.782243         NaN  ...  6598900  1435300   
2009-05-04   22.006989  10.010321         NaN  ...  5050000  2687900   
2009-05-05   21.434231   9.887935         NaN  ...  5925100  3689100   
...                ...        ...         ...  ...      ...      ...   
2020-06-04  101.979034  49.753910  218.039993  ...  2600700  1318100   
2020-06-05  105.032455  49.735439  219.550003  ...  1548000  1531100   
2020-06-08  108.803780  48.562477  219.000000  ...  1629800  1357900   
2020-06-09  105.731216  48.701019  220.720001  ...  2361900  1193800   
2020-06-10  101.615318  49.541485  223.679993  ...  1656300  1467300   

                                                                          \
                  VZ      WDC        WFC        XLF       XRT       YELP   
Date                                                                       
2009-04-29  21459256  4270400  142750100  192158977  26430400        NaN   
2009-04-30  23924686  7831600  115836200  215537513  29959000        NaN   
2009-05-01  13002285  4485000   96730100  137526705  32675200        NaN   
2009-05-04  16180380  4740400  294065000  260737864  31001000        NaN   
2009-05-05  17482986  4889500  175576000  200012388  17197400        NaN   
...              ...      ...        ...        ...       ...        ...   
2020-06-04  16147300  6973000   61839200   97575100   5712900  1081200.0   
2020-06-05  17105600  7708200  107651200  135859300   3879200  2572200.0   
2020-06-08  14963000  5376700   69051300   80298400   4900800  1194200.0   
2020-06-09  14530800  4124000   51480400   92833100   7829400  1653800.0   
2020-06-10  13649100  4894500   82090000  110848900   5895600   882800.0   

                           
                 YUM ZNGA  
Date                       
2009-04-29   7280216  NaN  
2009-04-30   7283276  NaN  
2009-05-01   6228203  NaN  
2009-05-04  10258764  NaN  
2009-05-05   8831181  NaN  
...              ...  ...  
2020-06-04   2537000  NaN  
2020-06-05   3722800  NaN  
2020-06-08   2748000  NaN  
2020-06-09   2063200  NaN  
2020-06-10   2418600  NaN  

[2799 rows x 600 columns]